In [20]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
matplotlib.style.use('ggplot')

import requests
import json

In [2]:
# specify data directory and file
data_dir = '/srv/zooniverse/raw_data/panoptes/all-panoptes-classifications-2017-01-30.csv'
# read csv file of classifications
classification_df = pd.read_csv(data_dir)
# create a project dataframe that contains the total number of classifications per project
project_df = classification_df.groupby('project_id').size().to_frame('total_classifications')

/home/sjm668/dev/zooniverse/src/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
project_df = project_df.reset_index()

In [28]:
base_url = r'https://panoptes.zooniverse.org/api/projects'
headers = {
    'Accept':'application/vnd.api+json; version=1',
    'Content-Type':'application/json'
}
params = {}
api_result_df = pd.DataFrame()
while True:
    r = requests.get(base_url,
                    params=params,
                    headers=headers)
    api_result = r.json()
    #print(json.dumps(api_result, indent=2))
    #break
    for project in api_result['projects']:
        api_result_df = api_result_df.append(pd.DataFrame([{
            'project_id':project['id'],
            'project_name':project['display_name'],
            'migrated':project['migrated'],
            'description':project['description'],
            'live':project['live'],
            'launch_date':project['launch_date'],
            'completeness':project['completeness'],
            'api_subject_count':project['subjects_count'],
            'api_classificaitons_count':project['classifiers_count']
        }]))
    
    if api_result['meta']['projects']['next_href']:
        params['page'] = api_result['meta']['projects']['next_href'].split('=')[-1]
        print(api_result['meta']['projects']['next_href'])
    else:
        break
api_result_df['project_id'] = pd.to_numeric(api_result_df['project_id'])
    

/projects?page=2
/projects?page=3
/projects?page=4
/projects?page=5
/projects?page=6
/projects?page=7
/projects?page=8
/projects?page=9
/projects?page=10
/projects?page=11
/projects?page=12
/projects?page=13
/projects?page=14
/projects?page=15
/projects?page=16
/projects?page=17
/projects?page=18
/projects?page=19
/projects?page=20
/projects?page=21
/projects?page=22
/projects?page=23
/projects?page=24
/projects?page=25


In [6]:
# project names included in both classification results and API
dump_api_project_overlap = api_result_df.loc[api_result_df['project_id'].isin(project_df['project_id'])]

In [7]:
len(api_result_df['project_name'].unique())

368

In [8]:
len(api_result_df['project_id'].unique())

372

In [9]:
# number of projects included in both classification dump and API results
len(api_result_df.loc[api_result_df['project_id'].isin(project_df['project_id'])])

311

In [10]:
# number of unique projects included in both classification dump and API results
len(api_result_df.loc[api_result_df['project_id'].isin(project_df['project_id'])]['project_id'].unique())

234

In [30]:
dump_api_project_overlap = dump_api_project_overlap.merge(project_df,on='project_id',how='outer')

In [31]:
dump_api_project_overlap

,project_id,project_name,total_classifications_x,total_classifications_y
0,4,Kitteh Zoo,4412.0,4412
1,4,Kitteh Zoo,4412.0,4412
2,17,Planet Four: Terrains,929108.0,929108
3,1,Snapshot Supernova,2021860.0,2021860
4,1,Snapshot Supernova,2021860.0,2021860
5,6,Wildebeest Watch,109756.0,109756
6,20,Monopole Quest!,2542.0,2542
7,3,Galaxy Zoo: Bar Lengths,319598.0,319598
8,3,Galaxy Zoo: Bar Lengths,319598.0,319598
9,11,Whales as Individuals,286871.0,286871


In [51]:
dump_api_project_no_overlap = api_result_df.loc[api_result_df['project_id'].isin(project_df['project_id'])]

,id,project_id,user_id,workflow_id,annotations,created_at,user_group_id,completed,gold_standard,expert_classifier,metadata,workflow_version,lifecycled_at
7542,10039982,376,429088.0,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-02-22 15:50:06.331265,2016-02-22 15:50:06.394891,t,NaN,NaN,"{""started_at"": ""2016-02-22T15:40:59+00:00"", ""u...",10.14,2016-02-22 15:50:06.386525
9130,9201103,376,NaN,205,"[{""task"": ""T0"", ""value"": true}]",2016-01-14 23:00:59.02662,2016-01-14 23:00:59.066003,t,NaN,NaN,"{""started_at"": ""2016-01-14T23:00:50+00:00"", ""u...",10.14,2016-01-14 23:00:59.060559
9488,16586105,376,3697.0,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-09-02 15:29:53.516373,2016-09-02 15:29:53.611471,t,NaN,NaN,"{""started_at"": ""2016-09-02T08:33:54-05:00"", ""u...",10.14,2016-09-02 15:29:53.601631
9495,9201087,376,NaN,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-01-14 23:00:49.684417,2016-01-14 23:00:49.750928,t,NaN,NaN,"{""started_at"": ""2016-01-14T22:53:10+00:00"", ""u...",10.14,2016-01-14 23:00:49.742751
11824,10865263,376,1872.0,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-03-28 23:58:42.999494,2016-03-28 23:58:43.111937,t,NaN,NaN,"{""started_at"": ""2016-03-29T00:13:11+01:00"", ""u...",10.14,2016-03-28 23:58:43.098303
14989,7038566,376,NaN,205,"[{""task"": ""T0"", ""value"": true}]",2016-01-13 13:38:02.659125,2016-01-13 13:38:22.765964,t,NaN,NaN,"{""started_at"": ""2016-01-13T13:37:56+00:00"", ""u...",10.14,2016-01-13 13:38:22.760548
17852,7038448,376,NaN,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-01-13 13:37:56.125778,2016-01-13 13:38:16.740468,t,NaN,NaN,"{""started_at"": ""2016-01-13T13:37:09+00:00"", ""u...",10.14,2016-01-13 13:38:16.731273
25411,7040252,376,184141.0,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-01-13 13:39:38.419139,2016-01-13 13:39:39.900211,t,NaN,NaN,"{""started_at"": ""2016-01-13T07:39:14-05:00"", ""u...",10.14,2016-01-13 13:39:39.878809
31745,11738394,376,NaN,205,"[{""task"": ""T0"", ""value"": true}]",2016-04-27 01:57:26.641504,2016-04-27 01:57:26.716699,t,NaN,NaN,"{""started_at"": ""2016-04-26T21:57:16-04:00"", ""u...",10.14,2016-04-27 01:57:26.701854
35214,12943503,376,3697.0,205,"[{""task"": ""T0"", ""value"": false}, {""task"": ""T2""...",2016-06-02 16:07:25.413415,2016-06-02 16:07:25.46284,t,NaN,NaN,"{""started_at"": ""2016-06-02T11:06:52-05:00"", ""u...",10.14,2016-06-02 16:07:25.458171


In [24]:
api_result_df.groupby('project_name')['project_name'].count().to_frame('count').sort_values('count',ascending=False)

,count
project_name,
Microplants,4
Test,3
Fauna Survey North East NSW with ABC North Coast Local Radio,2
Snapshots at Sea,2
Fossil Finder,2
GAMA Zoo,2
Space Warps,2
Galaxy Zoo,2
Galaxy Zoo - Mergers,2


In [29]:
api_result_df

,api_classificaitons_count,api_subject_count,completeness,description,launch_date,live,migrated,project_id,project_name
0,29859,0,None,Galaxy Zoo - Supernova,2010-03-26T19:25:48.000Z,True,True,25,Galaxy Zoo - Supernova
0,439,30,0,Classify cats. On the internet. For science.,None,False,False,4,Kitteh Zoo
0,11854,38089,0.489721,Help planetary scientists characterize surface...,2015-06-30T00:00:00.000Z,True,False,17,Planet Four: Terrains
0,128397,0,None,Find planets around stars. Lightcurve changes ...,2010-12-16T00:00:00.000Z,True,True,26,Planet Hunters
0,88547,0,None,Explore the surface of the Moon. We hope to st...,2010-05-11T00:00:00.000Z,True,True,22,Moon Zoo
0,0,0,0,Keep an eye on changing seasons!,None,False,False,18,Season Spotter BASIC Beta1
0,19265,0,None,Galaxy Zoo - Mergers,2009-11-23T13:48:45.000Z,True,True,24,Galaxy Zoo - Mergers
0,4464,74599,0.00010724,"Help in the hunt for supernovae, live!",2015-03-19T00:00:00.000Z,False,False,1,Snapshot Supernova
0,2636,5821,0.884513,Explore collective intelligence in wildebeest!,2015-06-30T00:00:00.000Z,True,False,6,Wildebeest Watch
0,11235,0,None,Ice Hunters,2011-08-09T00:00:00.000Z,True,True,29,Ice Hunters
